In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor, kernels
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# 観測データの初期選択
NSELECT = 2 # 観測データ初期選択数
RANDOM_STATE = 1 # 乱数SEED
# 獲得関数
V = 2 # v of sqrt(v t) of UCB
ACQ = "UCB" # TS or UCB
RANDOM_STATE_TS = 2 # TSの場合のreg.sample_y()の乱数SEED

In [ ]:
ROOT = ".."
df = pd.read_csv(f"{ROOT}/data_calculated/02x_m_sin5w.csv") # データ取得
print("df.shape:",df.shape)
DESCRIPTOR_NAMES = ['x1'] #,'x2'] # 説明変数、xの一次と二次。
TARGET_NAME = "y" # 目的変数

Xraw = df[DESCRIPTOR_NAMES].values # 生の説明変数
y = df[TARGET_NAME].values # 目的変数
print("Xraw.shape, y.shape:",Xraw.shape, y.shape)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(Xraw) # 規格化された説明変数
y = y - np.mean(y) # 可視化の都合で平均を引いておく


In [ ]:
plt.plot(X[:, 0], y, ".-")


In [ ]:
kernel = RBF(length_scale=1) # ガウス過程回帰のRBFカーネル
# k(x,y)=exp(-(x-y)^2/(2length_scale^2)), ユークリッド距離です。

# ガウス過程回帰の定義
# この例ではハイパーパラメタを最適化すると過学習するのでoptimizer=Noneとした。
reg = GaussianProcessRegressor(kernel=kernel, optimizer=None) 
# reg = GaussianProcessRegressor(kernel=kernel) 


In [ ]:
iopt = np.argmax(y) # yが最も大きいindexが得られる。
# インデックスioptが見つかったら終了とする。
print(f"argmax(X)={iopt}, max(X)={X[iopt]}") # 表示

In [ ]:
random.seed(RANDOM_STATE) # random.sampleに用いるSEED
idx = list(range(X.shape[0])) # Xのインデックスリスト
print("idx:", idx)
print("NSELECT:", NSELECT)
action = np.array(random.sample(idx, NSELECT)) 
# Xのインデックスリストから訓練データとして用いる初期値としてNSELECTつ選ぶ
print("initial action:", action)
print("initial X:", X[action].tolist()) # 表示を横に行うためにlistにする。

In [ ]:
from BO_misc import plot_GPR # user定義可視化関数を用いる。

print("Acquisition function:", ACQ)
final_action = False
for it in range(20):
    Xtrain = X[action] # 訓練データ
    ytrain = y[action] # 訓練データ
    reg.fit(Xtrain, ytrain) # モデル当てはめ
    yp_mean, yp_std = reg.predict(X, return_std=True) # 予測値平均値と標準偏差
    if ACQ=="UCB":
        acq = yp_mean + yp_std*np.sqrt(V*it) # UCB獲得関数
    elif ACQ == "TS":
        acq = reg.sample_y(X, random_state=RANDOM_STATE_TS+it)
    else:
        raise RutimeError(f"unknown ACQ={ACQ}")
    ia = np.argmax(acq) # 最大の獲得関数のインデックス
    action = np.hstack([action, ia]) # 訓練データインデックスリストに加える
    plot_GPR(X[:,0], y, Xtrain, ytrain, yp_mean, yp_std, acq, it, ia=ia) # 可視化
    # ravel()は一次元化するメンバー関数。
    if final_action: 
        break
    print(f"iteration {it+1}, next action {ia}, {X[ia]}")
    if iopt in action: # 最大yを与えるインデックスを含むか？
        print(f"iteration {it+1}, action={action}")
        final_action = True # 最大yの値を得るために、最後にもう一度行う。

In [ ]:
if ACQ != "TS":
    raise

# 多次元ガウス分布からサンプリングする。

In [ ]:
USE_SAMPLE_Y = True # True:ガウス過程.sample_y()を使う. False:multivariate_normal.rvs()を使う

random.seed(RANDOM_STATE) # random.sampleに用いるSEED
idx = list(range(X.shape[0])) # Xのインデックスリスト
action = np.array(random.sample(idx, NSELECT)) 

Xtrain = X[action] # 訓練データX
ytrain = y[action] # 訓練データy
reg.fit(Xtrain, ytrain) # 学習

if USE_SAMPLE_Y:
    yp_mean = reg.predict(X) # 平均値のみ
else:
    print("calculate covariance matrix")
    yp_mean, yp_covarencematrix = reg.predict(X, return_cov=True) # 平均値と共分散行列
    np.random.seed(seed=RANDOM_STATE_TS)

In [ ]:
from scipy.stats import multivariate_normal
fig, ax = plt.subplots()
# 50\vec{Y}点選択
for _i in range(50): 
    if USE_SAMPLE_Y:
        # random_stateの変数を変えないと、全て同じg_acqが得られる。
        acq = reg.sample_y(X, random_state=RANDOM_STATE_TS+_i)
    else:
        # 平均値と共分散行列を顕に用いる。
        acq = multivariate_normal.rvs(yp_mean, yp_covarencematrix)
    ax.plot(X[:, 0], acq, color="red", alpha=0.1)
ax.plot(X[:, 0], y, "--", color="blue")  # ,label="expriment")
ax.plot(Xtrain[:, 0], ytrain, "o", color="blue", label="expriment")
# 赤線が50 lines描かれる。
# 各赤線がStochastic samplingされた\vec{Y}